In [33]:
import pandas as pd
from rdflib import Graph, URIRef, Literal, RDF, RDFS, Namespace
from urllib.parse import quote
from rdflib.plugins.sparql import prepareQuery

## Cargar excel al grafo

In [34]:
dfcert = pd.read_excel('tramites/certificadosnormsinespaciocertlisto.xlsx')
dfdoc = pd.read_excel('tramites/certificadosnormsinespaciodoclisto.xlsx')

In [72]:
# Crear el grafo y el espacio de nombres
g = Graph()
ex = Namespace("http://example.org/")

# Diccionario para realizar un seguimiento de las instituciones
Ministerios = {}
Ministerios_uri = {}
Tramitesdic = {}
# Iterar sobre el DataFrame y agregar ministerio, instituciones y tramites con documentos
for _, row in dfcert.iterrows():
    tramite_nombre = row['Nombre del trámite']
    tramite_uri = Tramitesdic.get(tramite_nombre)
    if not tramite_uri:
        Ministerio_nombre = row['Ministerio']
        Ministerio_uri = Ministerios_uri.get(Ministerio_nombre)
        # Si el ministerio no existe en el diccionario, crear el nodo
        if not Ministerio_uri:
            Ministerio_uri = ex[quote(Ministerio_nombre.replace(" ", "_"))]
            g.add((Ministerio_uri, RDF.type, ex.Ministerio))
            Ministerios_uri[Ministerio_nombre] = Ministerio_uri
            Ministerios[Ministerio_nombre]={}
        institucion_nombre = row['Institución']
        institucion_uri = Ministerios[Ministerio_nombre].get(institucion_nombre)
        # Si la institución no existe en el diccionario, crear el nodo
        if not institucion_uri:
            institucion_uri = ex[quote(institucion_nombre.replace(" ", "_"))]
            g.add((institucion_uri, RDF.type, ex.Institucion))
            g.add((Ministerio_uri, ex.tieneInstitucion, institucion_uri))
            Ministerios[Ministerio_nombre][institucion_nombre] = institucion_uri        
        # Crear el nodo para el tramite y conectarlo a la institución
        tramite_nombre = row['Nombre del trámite']
        tramite_uri = ex[quote(tramite_nombre.replace(" ", "_"))]
        g.add((tramite_uri, RDF.type, ex.Tramite))
        g.add((institucion_uri, ex.realizaTramite, tramite_uri))
        Tramitesdic[tramite_nombre]=tramite_uri
    if row['¿Requiere de algún certificado?'] == 'Sí':
        # Crear nodos para cada documento y conectarlos al tramite
        certificado_nombre = row['Detalle certificados']
        certificado_uri = ex[quote(certificado_nombre.replace(" ", "_"))]
        g.add((certificado_uri, RDF.type, ex.certificado))
        g.add((tramite_uri, ex.requiereCertificado, certificado_uri))
    
#-------------------------Requerimientos-----------------
#-------Documentos notariales
    # Verificar si el tramite requiere documento
for _, row in dfdoc.iterrows():
    tramite_nombre = row['Nombre del trámite']
    tramite_uri = Tramitesdic.get(tramite_nombre)
    if row['¿Requiere de un documento notarial?'] == 'Sí':
        documento_nombre = row['Detalle documentos']
        documento_uri = ex[quote(documento_nombre.replace(" ","_"))]
        g.add((documento_uri, RDF.type, ex.Documento))
        g.add((tramite_uri, ex.requiereDocumento, documento_uri))

# Consultas

### tramites en cierto ministerio

In [4]:
query_instituciones_salud = """
    PREFIX ex: <http://example.org/>
    SELECT ?tramite
    WHERE {
        ?ministerio rdf:type ex:Ministerio.
        ?ministerio ex:tieneInstitucion ?institucion.
        ?institucion rdf:type ex:Institucion.
        ?institucion ex:realizaTramite ?tramite.
        FILTER (?ministerio = ex:Ministerio_Secretaria_General_de_Gobierno)

    }
"""

# Preparar la consulta
query_instituciones_salud = prepareQuery(query_instituciones_salud, initNs={"rdf": RDF, "ex": ex})

# Ejecutar la consulta en el grafo RDF
result_instituciones_salud = g.query(query_instituciones_salud)

# Imprimir los resultados
print("Instituciones asociadas al Ministerio de Salud:")
j=0
for row in result_instituciones_salud:
    j+=1
    print(row.tramite)
print(j)


Instituciones asociadas al Ministerio de Salud:
http://example.org/Sistema_Integrado_de_Atenci%C3%B3n_Ciudadana_%28SIAC%29_del_Consejo_Nacional_de_Televisi%C3%B3n_%28CNTV%29.
http://example.org/Recepci%C3%B3n_de_denuncias_sobre_programas_exhibidos_en_televisi%C3%B3n
http://example.org/Fondo_CNTV
http://example.org/Postulaci%C3%B3n_a_Concesiones_de_Televisi%C3%B3n_Digital_de_Libre_Recepci%C3%B3n
http://example.org/Solicitud_de_Acceso_a_la_Informaci%C3%B3n_P%C3%BAblica_%28Ley_20.285%29
http://example.org/Solicitud_de_Audiencia_%28Ley_20.730%29
http://example.org/Sistema_Integral_de_Atenci%C3%B3n_Ciudadana
http://example.org/Postulaci%C3%B3n_al_fondo_de_fortalecimiento_de_las_organizaciones_de_inter%C3%A9s_p%C3%BAblico
http://example.org/Postulaci%C3%B3n_al_fondo_al_fomento_de_medios_de_comunicaci%C3%B3n
http://example.org/Solicitud_de_Inscripci%C3%B3n_al_Catastro_de_Organizaciones_de_Inter%C3%A9s_P%C3%BAblico
http://example.org/Solicitud_de_Certificado_de_Organizaci%C3%B3n_Comunitaria
ht

### Ministerios

In [5]:
query_instituciones_salud = """
    SELECT DISTINCT ?ministerio
    WHERE {
        ?ministerio rdf:type ex:Ministerio ;

    }
"""
# Preparar la consulta
query_instituciones_salud = prepareQuery(query_instituciones_salud, initNs={"rdf": RDF, "ex": ex})

# Ejecutar la consulta en el grafo RDF
result_instituciones_salud = g.query(query_instituciones_salud)

# Imprimir los resultados
print("Instituciones asociadas al Ministerio de Salud:")
for row in result_instituciones_salud:
    print(row.ministerio)


Instituciones asociadas al Ministerio de Salud:
http://example.org/Ministerio_de_Transportes_y_Telecomunicaciones
http://example.org/Ministerio_de_Hacienda
http://example.org/Ministerio_de_las_Culturas%2C_las_Artes_y_el_Patrimonio
http://example.org/Consejo_de_Defensa_del_Estado
http://example.org/Ministerio_de_Desarrollo_Social_y_Familia
http://example.org/Ministerio_de_Agricultura
http://example.org/Ministerio_de_Econom%C3%ADa%2C_Fomento_y_Turismo
http://example.org/Ministerio_de_Defensa_Nacional
http://example.org/Ministerio_de_Obras_P%C3%BAblicas
http://example.org/Ministerio_del_Trabajo_y_Previsi%C3%B3n_Social
http://example.org/Ministerio_de_Justicia_y_Derechos_Humanos
http://example.org/Ministerio_de_Relaciones_Exteriores
http://example.org/Ministerio_del_Deporte
http://example.org/Ministerio_de_Salud
http://example.org/Ministerio_de_Educaci%C3%B3n
http://example.org/Ministerio_de_Energ%C3%ADa
http://example.org/Ministerio_del_Medio_Ambiente
http://example.org/Ministerio_de_Mine

### Instituciones asociadas a algún ministerio

In [6]:
query_instituciones_salud = """
    SELECT DISTINCT ?institucion
    WHERE {
        ?ministerio rdf:type ex:Ministerio ;
                    ex:tieneInstitucion ?institucion .
        ?institucion rdf:type ex:Institucion .
        FILTER (?ministerio = ex:Ministerio_de_Hacienda)
    }
"""
# Preparar la consulta
query_instituciones_salud = prepareQuery(query_instituciones_salud, initNs={"rdf": RDF, "ex": ex})

# Ejecutar la consulta en el grafo RDF
result_instituciones_salud = g.query(query_instituciones_salud)

# Imprimir los resultados
print("Instituciones asociadas al Ministerio de Hacienda:")
for row in result_instituciones_salud:
    print(row.institucion) 

Instituciones asociadas al Ministerio de Hacienda:
http://example.org/Servicio_Nacional_de_Aduanas
http://example.org/Direcci%C3%B3n_de_Compras_y_Contrataci%C3%B3n_P%C3%BAblica
http://example.org/Comisi%C3%B3n_para_el_Mercado_Financiero
http://example.org/Subsecretar%C3%ADa_de_Hacienda
http://example.org/Tesorer%C3%ADa_General_de_La_Rep%C3%BAblica
http://example.org/Superintendencia_de_Casinos_de_Juego
http://example.org/Direcci%C3%B3n_Nacional_del_Servicio_Civil
http://example.org/Unidad_de_An%C3%A1lisis_Financiero
http://example.org/Servicio_de_Impuestos_Internos
http://example.org/Unidad_Administradora_de_los_Tribunales_Tributarios_y_Aduaneros
http://example.org/Direcci%C3%B3n_de_Presupuestos


### Contar instuciones por ministerio

In [7]:
from rdflib import Graph, Namespace, RDF, URIRef, Literal
from urllib.parse import quote
# Consulta SPARQL para contar instituciones por ministerio
query_contar_instituciones_por_ministerio = """
    PREFIX ex: <http://example.org/>

    SELECT ?ministerio (COUNT( ?institucion) AS ?count)
    WHERE {
        ?ministerio rdf:type ex:Ministerio.
        ?ministerio ex:tieneInstitucion ?institucion.
    }
    GROUP BY ?ministerio
"""
# Ejecutar la consulta en el grafo RDF
result_contar_instituciones_por_ministerio = g.query(query_contar_instituciones_por_ministerio)

# Imprimir los resultados
print("Número de instituciones por ministerio:")
for row in result_contar_instituciones_por_ministerio:
    ministerio = row["ministerio"]
    cantidad_instituciones = row["count"]
    print(f"{ministerio}: {cantidad_instituciones}")

Número de instituciones por ministerio:
http://example.org/Ministerio_de_Transportes_y_Telecomunicaciones: 3
http://example.org/Ministerio_de_Hacienda: 11
http://example.org/Ministerio_de_las_Culturas%2C_las_Artes_y_el_Patrimonio: 3
http://example.org/Consejo_de_Defensa_del_Estado: 1
http://example.org/Ministerio_de_Desarrollo_Social_y_Familia: 8
http://example.org/Ministerio_de_Agricultura: 11
http://example.org/Ministerio_de_Econom%C3%ADa%2C_Fomento_y_Turismo: 14
http://example.org/Ministerio_de_Defensa_Nacional: 14
http://example.org/Ministerio_de_Obras_P%C3%BAblicas: 14
http://example.org/Ministerio_del_Trabajo_y_Previsi%C3%B3n_Social: 10
http://example.org/Ministerio_de_Justicia_y_Derechos_Humanos: 12
http://example.org/Ministerio_de_Relaciones_Exteriores: 6
http://example.org/Ministerio_del_Deporte: 2
http://example.org/Ministerio_de_Salud: 36
http://example.org/Ministerio_de_Educaci%C3%B3n: 18
http://example.org/Ministerio_de_Energ%C3%ADa: 4
http://example.org/Ministerio_del_Med

### Contar tramites por ministerio

In [8]:
from rdflib import Graph, Namespace, RDF, URIRef, Literal
from urllib.parse import quote

# Consulta SPARQL para contar instituciones por ministerio
query_contar_instituciones_por_ministerio = """
    PREFIX ex: <http://example.org/>

    SELECT ?ministerio (COUNT( ?tramite) AS ?count)
    WHERE {
        ?ministerio rdf:type ex:Ministerio.
        ?ministerio ex:tieneInstitucion ?institucion.
        ?institucion rdf:type ex:Institucion.
        ?institucion ex:realizaTramite ?tramite.
      
    }
    GROUP BY ?ministerio
"""
# Ejecutar la consulta en el grafo RDF
result_contar_instituciones_por_ministerio = g.query(query_contar_instituciones_por_ministerio)

# Imprimir los resultados
print("Número de tramites por ministerio:")
for row in result_contar_instituciones_por_ministerio:
    ministerio = row["ministerio"]
    cantidad_instituciones = row["count"]
    print(f"{ministerio}: {cantidad_instituciones}")

Número de tramites por ministerio:
http://example.org/Ministerio_de_Transportes_y_Telecomunicaciones: 175
http://example.org/Ministerio_de_Hacienda: 426
http://example.org/Ministerio_de_las_Culturas%2C_las_Artes_y_el_Patrimonio: 156
http://example.org/Consejo_de_Defensa_del_Estado: 4
http://example.org/Ministerio_de_Desarrollo_Social_y_Familia: 111
http://example.org/Ministerio_de_Agricultura: 176
http://example.org/Ministerio_de_Econom%C3%ADa%2C_Fomento_y_Turismo: 324
http://example.org/Ministerio_de_Defensa_Nacional: 374
http://example.org/Ministerio_de_Obras_P%C3%BAblicas: 84
http://example.org/Ministerio_del_Trabajo_y_Previsi%C3%B3n_Social: 239
http://example.org/Ministerio_de_Justicia_y_Derechos_Humanos: 158
http://example.org/Ministerio_de_Relaciones_Exteriores: 47
http://example.org/Ministerio_del_Deporte: 19
http://example.org/Ministerio_de_Salud: 525
http://example.org/Ministerio_de_Educaci%C3%B3n: 146
http://example.org/Ministerio_de_Energ%C3%ADa: 86
http://example.org/Minist

### Requisitos por ministerio

In [25]:
from rdflib import Graph, Literal, URIRef
from rdflib.namespace import RDF, XSD

# Ejecutar la consulta SPARQL
query = """
PREFIX ex: <http://example.org/>

SELECT ?ministerio (COUNT(DISTINCT ?documento) AS ?countDocumentos) (COUNT(DISTINCT?certificado) AS ?countCertificados) (COUNT(DISTINCT ?documento) + COUNT(DISTINCT ?certificado) AS ?suma)
WHERE {
    ?ministerio rdf:type ex:Ministerio.
    ?ministerio ex:tieneInstitucion ?institucion.
    ?institucion rdf:type ex:Institucion.
    ?institucion ex:realizaTramite ?tramite.

    OPTIONAL {
        ?tramite ex:requiereDocumento ?documento.
    }

    OPTIONAL {
        ?tramite ex:requiereCertificado ?certificado.
    }
}
GROUP BY ?ministerio
ORDER BY DESC(?suma)
"""
start_time = time.time()
result = g.query(query)
# Imprimir resultados
for row in result:
    ministerio, count_documentos, count_certificados, suma = row
    print(f"Ministerio: {ministerio}, Certificados: {count_certificados},Documentos: {count_documentos}, Suma: {suma}")
end_time = time.time()
elapsed_time = end_time - start_time
print("Tiempo de ejecución:", elapsed_time, "segundos")

Tiempo de ejecución: 0.642315149307251 segundos


### Requisitos asociados a un ministerio

In [16]:
query_instituciones_salud = """
    SELECT DISTINCT ?documento
    WHERE {
        ?ministerio rdf:type ex:Ministerio.
        ?ministerio ex:tieneInstitucion ?institucion .
        ?institucion rdf:type ex:Institucion .
        ?institucion ex:realizaTramite ?tramite.
        ?tramite ex:requiereDocumento ?documento.
        
        FILTER (?ministerio = ex:Ministerio_de_Econom%C3%ADa%2C_Fomento_y_Turismo)
    }
"""
# Preparar la consulta
query_instituciones_salud = prepareQuery(query_instituciones_salud, initNs={"rdf": RDF, "ex": ex})

# Ejecutar la consulta en el grafo RDF
result_instituciones_salud = g.query(query_instituciones_salud)

# Imprimir los resultados
print("requisitos asociadas al Ministerio de Hacienda:")
j=0
for row in result_instituciones_salud:
    j+=1    
    print(row.documento)
print(j)

requisitos asociadas al Ministerio de Hacienda:
http://example.org/Autorizaci%C3%B3n_notarial_de_uso_de_infraestructura
http://example.org/Mandato
http://example.org/autorizaci%C3%B3n_notarial_de_representante_legal
http://example.org/declaraci%C3%B3n_jurada_de_la_duraci%C3%B3n_de_la_sociedad
http://example.org/Fotocopia_notarial_de_los_estatutos_de_la_organizaci%C3%B3n_y_sus_modificaciones
http://example.org/Si_el_certificado_de_vigencia_no_establece_la_directiva%2C_se_deber%C3%A1_complementar_con_una_declaraci%C3%B3n_notarial_la_directiva%2C_por_el_presidente_o_representante_legal_de_la_organizaci%C3%B3n.
http://example.org/poder_de_acreditaci%C3%B3n_de_representaci%C3%B3n_legal_vigente%2C_en_caso_de_personas_jur%C3%ADdicas
http://example.org/declaraci%C3%B3n_jurada_de_residencia
http://example.org/Declaraci%C3%B3n_jurada_de_origen_y
http://example.org/o_informes_de_toxinas_emitidos_por_el_Servicio_de_Salud%2C_seg%C3%BAn_corresponda.
http://example.org/Autorizaci%C3%B3n_notarial_en_c

### Número de requisitos distintos

In [29]:
ex = Namespace("http://example.org/")
# Definir la consulta SPARQL
query = prepareQuery("""
    PREFIX ex: <http://example.org/>

    SELECT (COUNT(DISTINCT ?documento) AS ?numDocumentos) (COUNT(DISTINCT ?certificado) AS ?numCertificados)
    WHERE {
      ?tramite rdf:type ex:Tramite.
      OPTIONAL { ?tramite ex:requiereDocumento ?documento. }
      OPTIONAL { ?tramite ex:requiereCertificado ?certificado. }
    }
""", initNs={"ex": ex})
start_time = time.time()

# Ejecutar la consulta y obtener los resultados
resultados = g.query(query)

# Detener temporizador y calcular tiempo transcurrido
end_time = time.time()

elapsed_time = end_time - start_time
# Mostrar los resultados
for resultado in resultados:
    num_documentos = resultado[0]
    num_certificados = resultado[1]
    print("Número de documentos distintos:", num_documentos)
    print("Número de certificados distintos:", num_certificados)
    
print("Tiempo de ejecución:", elapsed_time, "segundos")

Número de documentos distintos: 365
Número de certificados distintos: 93
1
Tiempo de ejecución: 0.0 segundos


# ----------------------
# Modificaciones al grafo
# ----------------------

### Consultar si se agregó el nuevo nodo

In [79]:
ex = Namespace("http://example.org/")

# Definir la consulta SPARQL para obtener la información de una institución específica
query = prepareQuery("""
    PREFIX ex: <http://example.org/>

    SELECT ?institucion ?tramite
    WHERE {
        ?institucion rdf:type ex:Institucion .
        ?institucion ex:tieneNuevoNodo ?tramite .
        FILTER(?institucion = ex:Servicio_Nacional_de_Aduanas)  # Filtrar por la institución específica
    }
""", initNs={"ex": ex})

# Ejecutar la consulta
results = g.query(query)

# Imprimir los resultados
for row in results:
    print(row)

## Agregar un nuevo nodo a cada institucion

In [75]:
from rdflib import Graph, Namespace, URIRef
from rdflib.namespace import RDF

start_time = time.time()
# Definir el prefijo y el espacio de nombres
# Definir la consulta SPARQL para seleccionar todos los sujetos de un cierto tipo
query = """
    PREFIX ex: <http://example.org/>
    SELECT ?institucion
    WHERE {
        ?institucion rdf:type ex:Institucion.
    }
"""
# Ejecutar la consulta SPARQL
result = g.query(query)
end_time = time.time()

elapsed_time = end_time - start_time
print("Tiempo de ejecución:", elapsed_time, "segundos")

Tiempo de ejecución: 0.005017280578613281 segundos


In [76]:
start_time = time.time()

# Recorrer los resultados y agregar un nuevo nodo a cada uno
for row in result:
    # Obtener el sujeto de la fila
    sujeto = row[0]
    
    # Agregar el nuevo nodo al sujeto
    nuevo_nodo = URIRef("http://example.org/NuevoNodo")
    g.add((sujeto, ex.tieneNuevoNodo, ex.nuevo_nodo))
    g.add((ex.nuevo_nodo, RDF.type, ex.newnodo))
end_time = time.time()
elapsed_time = end_time - start_time
print("Tiempo de ejecución:", elapsed_time, "segundos")

Tiempo de ejecución: 0.012992620468139648 segundos


### eliminar nuevo nodo en institución

In [ ]:
start_time = time.time()

# Buscar todos los nodos que agregaste previamente y eliminarlos
nodos_agregados = g.subjects(predicate=ex.tieneNuevoNodo, object=ex.nuevo_nodo)
for nodo in nodos_agregados:
    g.remove((nodo, None, None))  # Eliminar todas las tripletas relacionadas con el nodo
    g.remove((None, None, nodo))  # Eliminar todas las tripletas donde el nodo es objeto

end_time = time.time()
elapsed_time = end_time - start_time
print("Tiempo de ejecución:", elapsed_time, "segundos")

## Agregar nuevo nodo a cada tramite

In [ ]:
start_time = time.time()
# Definir el prefijo y el espacio de nombres
# Definir la consulta SPARQL para seleccionar todos los sujetos de un cierto tipo
query = """
    PREFIX ex: <http://example.org/>
    SELECT ?tramite
    WHERE {
        ?institucion rdf:type ex:Tramite.
    }
"""
# Ejecutar la consulta SPARQL
resulttramite = g.query(query)
end_time = time.time()

elapsed_time = end_time - start_time
print("Tiempo de ejecución:", elapsed_time, "segundos")

In [ ]:
start_time = time.time()

# Recorrer los resultados y agregar un nuevo nodo a cada uno
for row in resulttramite:
    # Obtener el sujeto de la fila
    sujeto = row[0]
    
    # Agregar el nuevo nodo al sujeto
    nuevo_nodo = URIRef("http://example.org/NuevoNodo")
    g.add((sujeto, ex.tieneNuevoNodo, ex.nuevo_nodo))
    g.add((ex.nuevo_nodo, RDF.type, ex.newnodo))
end_time = time.time()
elapsed_time = end_time - start_time
print("Tiempo de ejecución:", elapsed_time, "segundos")

### eliminar nuevo nodo en tramite

In [ ]:
start_time = time.time()

# Buscar todos los nodos que agregaste previamente y eliminarlos
nodos_agregados = g.subjects(predicate=ex.tieneNuevoNodo, object=ex.nuevo_nodo)
for nodo in nodos_agregados:
    g.remove((nodo, None, None))  # Eliminar todas las tripletas relacionadas con el nodo
    g.remove((None, None, nodo))  # Eliminar todas las tripletas donde el nodo es objeto

end_time = time.time()
elapsed_time = end_time - start_time
print("Tiempo de ejecución:", elapsed_time, "segundos")


# Eliminacion de nodos

In [78]:
start_time = time.time()

# Buscar todos los nodos que agregaste previamente y eliminarlos
nodos_agregados = g.subjects(predicate=ex.tieneNuevoNodo, object=ex.nuevo_nodo)
for nodo in nodos_agregados:
    g.remove((nodo, None, None))  # Eliminar todas las tripletas relacionadas con el nodo
    g.remove((None, None, nodo))  # Eliminar todas las tripletas donde el nodo es objeto

end_time = time.time()
elapsed_time = end_time - start_time
print("Tiempo de ejecución:", elapsed_time, "segundos")


Tiempo de ejecución: 0.04686427116394043 segundos


### Comprovaciones

### Número de instituciones por ministerio

In [7]:
import pandas as pd

# Lee el archivo Excel
df = pd.read_excel('tramites/tramites_disponibles.xlsx')  # Reemplaza 'tu_archivo.xlsx' con la ruta de tu archivo

# Filtra las filas donde la columna 'Ministerio' es igual a 'Ministerio de Salud'
df_filtrado = df[df['Ministerio'] == 'Ministerio de Salud']

# Obtiene una lista única de instituciones
lista_instituciones = df_filtrado['Institución'].unique()

# Obtiene el número total de instituciones únicas
numero_instituciones = len(lista_instituciones)

# Muestra el número de instituciones
print("Número total de instituciones únicas:", numero_instituciones)


Número total de instituciones únicas: 36


### Número de tramites

In [56]:
import pandas as pd

# Lee el archivo Excel
df = pd.read_excel('tramites/tramites_disponibles.xlsx')  # Reemplaza 'tu_archivo.xlsx' con la ruta de tu archivo

# Filtra las filas donde la columna 'Ministerio' es igual a 'Ministerio de Salud'
df_filtrado = df[df['Ministerio'] == 'Ministerio de Salud']

# Obtiene una lista única de instituciones


# Obtiene el número total de instituciones únicas
numero_instituciones = len(df_filtrado)

# Muestra el número de instituciones
print("Número total de instituciones únicas:", numero_instituciones)

Número total de instituciones únicas: 525
